# 07 — Live Inference
Generate real-time trade signals using the latest trained model.
Loads the most recent walk-forward fold's artifacts and runs the full pipeline.

In [ ]:
!pip install -q torch xgboost ccxt PyWavelets pandas-ta hmmlearn numba scikit-learn pyyaml tqdm pyarrow

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import sys, os, json
REPO_DIR = '/content/scalp2'
if not os.path.exists(REPO_DIR):
    !git clone https://github.com/<YOUR_USERNAME>/scalp2.git {REPO_DIR}
sys.path.insert(0, REPO_DIR)

import logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s %(name)s %(levelname)s: %(message)s')

import numpy as np
import pandas as pd
import torch
from datetime import datetime, timezone

from scalp2.config import load_config
config = load_config(f'{REPO_DIR}/config.yaml')

CHECKPOINT_DIR = '/content/drive/MyDrive/scalp2/checkpoints'

In [ ]:
from scalp2.models.hybrid import HybridEncoder
from scalp2.models.meta_learner import XGBoostMetaLearner
from scalp2.regime.hmm import RegimeDetector
from scalp2.execution.signal_generator import SignalGenerator
from scalp2.utils.serialization import load_fold_artifacts

# Find latest fold
fold_dirs = sorted([d for d in os.listdir(CHECKPOINT_DIR) if d.startswith('fold_')])
latest_fold_idx = int(fold_dirs[-1].split('_')[1])
print(f'Loading latest fold: {latest_fold_idx}')

# Load artifacts
artifacts = load_fold_artifacts(CHECKPOINT_DIR, latest_fold_idx)

with open(f'/content/drive/MyDrive/scalp2/data/processed/feature_columns.json', 'r') as f:
    feature_cols = json.load(f)

n_features = len(feature_cols)

# Reconstruct models
model = HybridEncoder(n_features, config.model)
model.load_state_dict(artifacts['model_state'])

meta_learner = XGBoostMetaLearner(config.model.xgboost)
xgb_path = os.path.join(CHECKPOINT_DIR, f'xgb_fold_{latest_fold_idx:03d}.json')
meta_learner.load(xgb_path)

regime_detector = artifacts.get('regime_detector', RegimeDetector(config.regime))
scaler = artifacts['scaler']
top_feature_indices = artifacts['top_feature_indices']

print(f'Model loaded: {model.count_parameters():,} params')

In [ ]:
# Initialize signal generator
signal_gen = SignalGenerator(
    config=config,
    model=model,
    meta_learner=meta_learner,
    regime_detector=regime_detector,
    scaler=scaler,
    top_feature_indices=top_feature_indices,
)
print('Signal generator initialized')

In [ ]:
# Fetch latest data and generate signal
from scalp2.data.downloader import OHLCVDownloader
from scalp2.features.builder import build_features, get_feature_columns
from scalp2.data.preprocessing import clean_ohlcv

# Fetch last ~100 bars to ensure enough for seq_len + warmup
import ccxt
exchange = ccxt.binanceusdm({'enableRateLimit': True})
bars_needed = config.model.seq_len + 300  # Extra for feature warmup

ohlcv = exchange.fetch_ohlcv('BTC/USDT', '15m', limit=bars_needed)
df_live = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
df_live['timestamp'] = pd.to_datetime(df_live['timestamp'], unit='ms', utc=True)
df_live = df_live.set_index('timestamp')
df_live = df_live.astype(np.float32)

# Build features
df_live_feat = build_features(df_live, config.features)

# Get current values
current_atr = df_live_feat[f'atr_{config.labeling.atr_period}'].iloc[-1]
current_price = df_live_feat['close'].iloc[-1]
current_time = df_live_feat.index[-1].to_pydatetime()

# Scale and prepare window
live_features = df_live_feat[feature_cols].values
live_scaled = scaler.transform(live_features).astype(np.float32)
window = live_scaled[-config.model.seq_len:]

# Generate signal
signal = signal_gen.generate(
    features_scaled=window,
    regime_df=df_live_feat.iloc[-config.model.seq_len:],
    current_atr=current_atr,
    current_price=current_price,
    current_time=current_time,
)

print(f'\n=== TRADE SIGNAL ===')
print(f'Direction: {signal.direction.value}')
print(f'Confidence: {signal.confidence:.3f}')
print(f'Entry: {signal.entry_price:.2f}')
print(f'TP: {signal.take_profit:.2f}')
print(f'SL: {signal.stop_loss:.2f}')
print(f'Size: {signal.position_size:.4f}')
print(f'Regime: {signal.regime}')
print(f'Probabilities: {signal.probabilities}')